In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoderEstimator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from collections import defaultdict
from pyspark.ml.feature import Imputer
from pyspark.sql.functions import col,sum
import datetime
from pyspark.ml import PipelineModel

In [2]:
def get_types(df):
    print(df.schema)
    print(df.schema.fields)
    data_types_map = defaultdict(list)
    for entry in df.schema.fields:
      data_types_map[str(entry.dataType)].append(entry.name)
    print(data_types_map)
    return data_types_map

def filter_rows_with_missing_labels(df, target):
    #print(df.count())
    tmp = df.filter(df[target].isNotNull())
    print(tmp.count())
    return tmp
  
def drop_features(df, features):
    print(len(df.columns))
    for feature in features:
      df = df.drop(feature)
    print(len(df.columns))
    return df

def filter_features_with_missing_data(df):
    tmp = df.filter(df[target].isNotNull())
    return tmp
  
def get_missing_info(df):
    tmp = df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns))
    display(tmp)

def eda_categorical_feature(df, feature):
    tmp = df.groupby(feature).count().toPandas()
    print(tmp)
        
def eda_continuous_features(df, feature):
    df.describe(feature).show()
  
def cast_integer_features_as_double(df, data_types_map):
    for c in data_types_map["IntegerType"]:
       df = df.withColumn(c, df[c].cast('double'))
    df.printSchema()
    return df
        
def imputer_continuous_features(df, data_types_map):
    continuous_features = list(set(data_types_map['DoubleType']) - set(['DEP_DEL15']))
    continuous_features_imputed = [var + "_imputed" for var in  continuous_features]
    imputer = Imputer(inputCols = continuous_features, outputCols = continuous_features_imputed)
    tmp = imputer.fit(df).transform(df)
    get_missing_info(tmp)
    return [imputer]

def impute_categorical_features(df, data_types_map):
    missing_data_fill = {}
    for var in data_types_map['StringType']:
      missing_data_fill[var] = "missing"
    tmp = df.fillna(missing_data_fill)
    get_missing_info(tmp)
    return tmp
    
def encoder_categorical_features(df, data_types_map):
    categorical_features = list(set(data_types_map['StringType']) - set(['_c46']))
    string_indexers = [StringIndexer(inputCol=feature, outputCol=feature+"_index") for feature in categorical_features]

    ohe_input_features = [ feature+"_index" for feature in categorical_features]
    ohe_output_features = [ feature+"_encoded" for feature in categorical_features]
    ohe_encoder = OneHotEncoderEstimator(inputCols=ohe_input_features, outputCols=ohe_output_features)  
    pipeline = Pipeline(stages=string_indexers + [ohe_encoder])
    tmp = pipeline.fit(df).transform(df)
    tmp.printSchema()
    return string_indexers + [ohe_encoder]

def assembler_for_feature_vector(df, data_types_map):
    features = []
    #categorical_features = list(set(data_types_map['StringType']) - set(['_c46']))
    #features = features + [ feature+"_encoded" for feature in categorical_features]
    
    continuous_features = list(set(data_types_map['DoubleType']) - set(['DEP_DEL15']))
    features = features + [var + "_imputed" for var in  continuous_features]
    print(features)
    assembler = VectorAssembler(inputCols= features, outputCol="features")
    #tmp = assembler.transform(df)
    #tmp.printSchema()
    return [assembler]

def split_data(df, train_percent):
    return df.randomSplit([train_percent, 1-train_percent], seed=123);

def build_and_tune_model_with_cv(pipeline, param_grid, evaluatior, df):
    # define grid based cross validator
    crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=param_grid,
                          evaluator= evaluator,
                          numFolds=2)

    # train model using fit
    cv_model = crossval.fit(df)
    return cv_model
  
def model_summary_lr(cv_model):
    print(cv_model)
    print(cv_model.avgMetrics)
    best_model = cv_model.bestModel
    print(best_model)
    print(best_model.stages)
    lr_model = best_model.stages[-1]
    print(lr_model)
    #print(lr_model.featureImportances)
    print(lr_model.coefficients)
    print(lr_model.intercept)
  
def predict(cv_model, evaluator, df):
    # evaluate model on test set
    predictions = cv_model.transform(df)
    print(predictions)
    predictions.printSchema()
    print(evaluator.evaluate(predictions))

def persist_model(cv_model, type, model_dir):
    datestamp = datetime.datetime.now().strftime('%m-%d-%Y-%s');
    file_name = "cv_model_" + type + "_" + datestamp
    path = model_dir + file_name
    print(path)
    cv_model.bestModel.save(path)
    
def load_model(path):
    return PipelineModel.load(path)



In [3]:
airline_delays = spark.read.load("/FileStore/tables/airline_delays.csv",
                             format="csv", header="true", inferSchema="true", sep=",")
airline_delays.printSchema()


root
-- YEAR: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY_OF_MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: timestamp (nullable = true)
-- UNIQUE_CARRIER: string (nullable = true)
-- TAIL_NUM: string (nullable = true)
-- FL_NUM: integer (nullable = true)
-- ORIGIN_AIRPORT_ID: integer (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- DEST_AIRPORT_ID: integer (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_TIME: integer (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: integer (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- ARR_TIME: integer (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: integer (nullable = true)
-- CANCELLED: double (nullable = true)
-- CANCELLATION_CODE: string (nullable = true)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: integer (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = true)
-- LATE_AIRCRAFT_DELAY: double (nullable = true)
-- _c46: string (nullable = true)

In [4]:
display(airline_delays)

YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,_c46
2018,1,2,2,2018-01-02T00:00:00.000+0000,WN,N282WN,1325,14843,SJU,"San Juan, PR",PR,13204,MCO,"Orlando, FL",FL,1105,1100,-5.0,0.0,0.0,-1,15.0,1115,1303,6.0,1315,1309,-6.0,0.0,0.0,-1,0.0,null,0.0,190.0,189.0,168.0,1.0,1189.0,5,null,null,null,null,null,null
2018,1,2,2,2018-01-02T00:00:00.000+0000,WN,N8686A,5159,14843,SJU,"San Juan, PR",PR,13204,MCO,"Orlando, FL",FL,1800,1752,-8.0,0.0,0.0,-1,12.0,1804,1949,6.0,2010,1955,-15.0,0.0,0.0,-1,0.0,null,0.0,190.0,183.0,165.0,1.0,1189.0,5,null,null,null,null,null,null
2018,1,2,2,2018-01-02T00:00:00.000+0000,WN,N430WN,5890,14843,SJU,"San Juan, PR",PR,13204,MCO,"Orlando, FL",FL,1545,1601,16.0,16.0,1.0,1,14.0,1615,1759,5.0,1755,1804,9.0,9.0,0.0,0,0.0,null,0.0,190.0,183.0,164.0,1.0,1189.0,5,null,null,null,null,null,null
2018,1,2,2,2018-01-02T00:00:00.000+0000,WN,N499WN,6618,14843,SJU,"San Juan, PR",PR,13204,MCO,"Orlando, FL",FL,600,553,-7.0,0.0,0.0,-1,14.0,607,751,6.0,810,757,-13.0,0.0,0.0,-1,0.0,null,0.0,190.0,184.0,164.0,1.0,1189.0,5,null,null,null,null,null,null
2018,1,2,2,2018-01-02T00:00:00.000+0000,WN,N8522P,1701,14843,SJU,"San Juan, PR",PR,13232,MDW,"Chicago, IL",IL,1620,1614,-6.0,0.0,0.0,-1,11.0,1625,1908,25.0,1925,1933,8.0,8.0,0.0,0,0.0,null,0.0,305.0,319.0,283.0,1.0,2057.0,9,null,null,null,null,null,null
2018,1,2,2,2018-01-02T00:00:00.000+0000,WN,N8639B,844,14843,SJU,"San Juan, PR",PR,15304,TPA,"Tampa, FL",FL,1715,1737,22.0,22.0,1.0,1,14.0,1751,1943,5.0,1925,1948,23.0,23.0,1.0,1,0.0,null,0.0,190.0,191.0,172.0,1.0,1237.0,5,0.0,0.0,1.0,0.0,22.0,null
2018,1,2,2,2018-01-02T00:00:00.000+0000,WN,N8530W,4679,14843,SJU,"San Juan, PR",PR,15304,TPA,"Tampa, FL",FL,1445,1441,-4.0,0.0,0.0,-1,18.0,1459,1648,3.0,1655,1651,-4.0,0.0,0.0,-1,0.0,null,0.0,190.0,190.0,169.0,1.0,1237.0,5,null,null,null,null,null,null
2018,1,2,2,2018-01-02T00:00:00.000+0000,WN,N205WN,6294,14869,SLC,"Salt Lake City, UT",UT,10800,BUR,"Burbank, CA",CA,1950,2019,29.0,29.0,1.0,1,15.0,2034,2103,2.0,2045,2105,20.0,20.0,1.0,1,0.0,null,0.0,115.0,106.0,89.0,1.0,574.0,3,1.0,0.0,0.0,0.0,19.0,null
2018,1,2,2,2018-01-02T00:00:00.000+0000,WN,N720WN,5245,14869,SLC,"Salt Lake City, UT",UT,11259,DAL,"Dallas, TX",TX,535,533,-2.0,0.0,0.0,-1,25.0,558,900,8.0,925,908,-17.0,0.0,0.0,-2,0.0,null,0.0,170.0,155.0,122.0,1.0,999.0,4,null,null,null,null,null,null
2018,1,2,2,2018-01-02T00:00:00.000+0000,WN,N222WN,2278,14869,SLC,"Salt Lake City, UT",UT,11292,DEN,"Denver, CO",CO,1245,1242,-3.0,0.0,0.0,-1,21.0,1303,1357,5.0,1405,1402,-3.0,0.0,0.0,-1,0.0,null,0.0,80.0,80.0,54.0,1.0,391.0,2,null,null,null,null,null,null


In [5]:
print(airline_delays.count())
data_types_map = get_types(airline_delays)

570118
StructType(List(StructField(YEAR,IntegerType,true),StructField(MONTH,IntegerType,true),StructField(DAY_OF_MONTH,IntegerType,true),StructField(DAY_OF_WEEK,IntegerType,true),StructField(FL_DATE,TimestampType,true),StructField(UNIQUE_CARRIER,StringType,true),StructField(TAIL_NUM,StringType,true),StructField(FL_NUM,IntegerType,true),StructField(ORIGIN_AIRPORT_ID,IntegerType,true),StructField(ORIGIN,StringType,true),StructField(ORIGIN_CITY_NAME,StringType,true),StructField(ORIGIN_STATE_ABR,StringType,true),StructField(DEST_AIRPORT_ID,IntegerType,true),StructField(DEST,StringType,true),StructField(DEST_CITY_NAME,StringType,true),StructField(DEST_STATE_ABR,StringType,true),StructField(CRS_DEP_TIME,IntegerType,true),StructField(DEP_TIME,IntegerType,true),StructField(DEP_DELAY,DoubleType,true),StructField(DEP_DELAY_NEW,DoubleType,true),StructField(DEP_DEL15,DoubleType,true),StructField(DEP_DELAY_GROUP,IntegerType,true),StructField(TAXI_OUT,DoubleType,true),StructField(WHEELS_OFF,IntegerType,true),StructField(WHEELS_ON,IntegerType,true),StructField(TAXI_IN,DoubleType,true),StructField(CRS_ARR_TIME,IntegerType,true),StructField(ARR_TIME,IntegerType,true),StructField(ARR_DELAY,DoubleType,true),StructField(ARR_DELAY_NEW,DoubleType,true),StructField(ARR_DEL15,DoubleType,true),StructField(ARR_DELAY_GROUP,IntegerType,true),StructField(CANCELLED,DoubleType,true),StructField(CANCELLATION_CODE,StringType,true),StructField(DIVERTED,DoubleType,true),StructField(CRS_ELAPSED_TIME,DoubleType,true),StructField(ACTUAL_ELAPSED_TIME,DoubleType,true),StructField(AIR_TIME,DoubleType,true),StructField(FLIGHTS,DoubleType,true),StructField(DISTANCE,DoubleType,true),StructField(DISTANCE_GROUP,IntegerType,true),StructField(CARRIER_DELAY,DoubleType,true),StructField(WEATHER_DELAY,DoubleType,true),StructField(NAS_DELAY,DoubleType,true),StructField(SECURITY_DELAY,DoubleType,true),StructField(LATE_AIRCRAFT_DELAY,DoubleType,true),StructField(_c46,StringType,true)))
[StructField(YEAR,IntegerType,true), StructField(MONTH,IntegerType,true), StructField(DAY_OF_MONTH,IntegerType,true), StructField(DAY_OF_WEEK,IntegerType,true), StructField(FL_DATE,TimestampType,true), StructField(UNIQUE_CARRIER,StringType,true), StructField(TAIL_NUM,StringType,true), StructField(FL_NUM,IntegerType,true), StructField(ORIGIN_AIRPORT_ID,IntegerType,true), StructField(ORIGIN,StringType,true), StructField(ORIGIN_CITY_NAME,StringType,true), StructField(ORIGIN_STATE_ABR,StringType,true), StructField(DEST_AIRPORT_ID,IntegerType,true), StructField(DEST,StringType,true), StructField(DEST_CITY_NAME,StringType,true), StructField(DEST_STATE_ABR,StringType,true), StructField(CRS_DEP_TIME,IntegerType,true), StructField(DEP_TIME,IntegerType,true), StructField(DEP_DELAY,DoubleType,true), StructField(DEP_DELAY_NEW,DoubleType,true), StructField(DEP_DEL15,DoubleType,true), StructField(DEP_DELAY_GROUP,IntegerType,true), StructField(TAXI_OUT,DoubleType,true), StructField(WHEELS_OFF,IntegerType,true), StructField(WHEELS_ON,IntegerType,true), StructField(TAXI_IN,DoubleType,true), StructField(CRS_ARR_TIME,IntegerType,true), StructField(ARR_TIME,IntegerType,true), StructField(ARR_DELAY,DoubleType,true), StructField(ARR_DELAY_NEW,DoubleType,true), StructField(ARR_DEL15,DoubleType,true), StructField(ARR_DELAY_GROUP,IntegerType,true), StructField(CANCELLED,DoubleType,true), StructField(CANCELLATION_CODE,StringType,true), StructField(DIVERTED,DoubleType,true), StructField(CRS_ELAPSED_TIME,DoubleType,true), StructField(ACTUAL_ELAPSED_TIME,DoubleType,true), StructField(AIR_TIME,DoubleType,true), StructField(FLIGHTS,DoubleType,true), StructField(DISTANCE,DoubleType,true), StructField(DISTANCE_GROUP,IntegerType,true), StructField(CARRIER_DELAY,DoubleType,true), StructField(WEATHER_DELAY,DoubleType,true), StructField(NAS_DELAY,DoubleType,true), StructField(SECURITY_DELAY,DoubleType,true), StructField(LATE_AIRCRAFT_DELAY,DoubleType,true), StructField(_c46,StringType,true)]
defaultdict(<class 'list'>, {'StringType': 

In [6]:
airline_delays = cast_integer_features_as_double(airline_delays, data_types_map)

root
-- YEAR: double (nullable = true)
-- MONTH: double (nullable = true)
-- DAY_OF_MONTH: double (nullable = true)
-- DAY_OF_WEEK: double (nullable = true)
-- FL_DATE: timestamp (nullable = true)
-- UNIQUE_CARRIER: string (nullable = true)
-- TAIL_NUM: string (nullable = true)
-- FL_NUM: double (nullable = true)
-- ORIGIN_AIRPORT_ID: double (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- DEST_AIRPORT_ID: double (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- CRS_DEP_TIME: double (nullable = true)
-- DEP_TIME: double (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: double (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: double (nullable = true)
-- WHEELS_ON: double (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: double (nullable = true)
-- ARR_TIME: double (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: double (nullable = true)
-- CANCELLED: double (nullable = true)
-- CANCELLATION_CODE: string (nullable = true)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: double (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = true)
-- LATE_AIRCRAFT_DELAY: double (nullable = true)
-- _c46: string (nullable = true)

In [7]:
airline_delays.printSchema()

root
-- YEAR: double (nullable = true)
-- MONTH: double (nullable = true)
-- DAY_OF_MONTH: double (nullable = true)
-- DAY_OF_WEEK: double (nullable = true)
-- FL_DATE: timestamp (nullable = true)
-- UNIQUE_CARRIER: string (nullable = true)
-- TAIL_NUM: string (nullable = true)
-- FL_NUM: double (nullable = true)
-- ORIGIN_AIRPORT_ID: double (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- DEST_AIRPORT_ID: double (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- CRS_DEP_TIME: double (nullable = true)
-- DEP_TIME: double (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: double (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: double (nullable = true)
-- WHEELS_ON: double (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: double (nullable = true)
-- ARR_TIME: double (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: double (nullable = true)
-- CANCELLED: double (nullable = true)
-- CANCELLATION_CODE: string (nullable = true)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: double (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = true)
-- LATE_AIRCRAFT_DELAY: double (nullable = true)
-- _c46: string (nullable = true)

In [8]:
target = 'DEP_DEL15'
eda_categorical_feature(airline_delays, target)

DEP_DEL15 count
0 0.0 453644
1 NaN 17323
2 1.0 99151

In [9]:
airline_delays_not_null = filter_rows_with_missing_labels(airline_delays, target)


552795

In [10]:
eda_categorical_feature(airline_delays_not_null, target)

DEP_DEL15 count
0 0.0 453644
1 1.0 99151

In [11]:
#split the data into train and validation sets
airline_delays_train, airline_delays_test = split_data(airline_delays_not_null, 0.75)

#persist the train and test frames
airline_delays_train.persist(); 
print(airline_delays_train.count());
airline_delays_test.persist(); 
print(airline_delays_test.count());


414637
138158

In [12]:
get_missing_info(airline_delays_train)

YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,_c46
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,172,172,543,543,0,543,1373,1373,1373,1373,0,414382,0,0,1233,1233,0,0,0,341223,341223,341223,341223,341223,414637


In [13]:
#impute categorical features
airline_delays_train = impute_categorical_features(airline_delays_train, data_types_map)

YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,_c46
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,172,172,543,543,0,543,1373,1373,1373,1373,0,0,0,0,1233,1233,0,0,0,341223,341223,341223,341223,341223,0


In [14]:
#imputer for continuous features
imputer = imputer_continuous_features(airline_delays_train, data_types_map)

YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,TAIL_NUM,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,_c46,ACTUAL_ELAPSED_TIME_imputed,AIR_TIME_imputed,CANCELLED_imputed,ARR_DELAY_NEW_imputed,FLIGHTS_imputed,SECURITY_DELAY_imputed,DEP_DELAY_imputed,ARR_DEL15_imputed,NAS_DELAY_imputed,DIVERTED_imputed,TAXI_IN_imputed,CRS_ELAPSED_TIME_imputed,CARRIER_DELAY_imputed,LATE_AIRCRAFT_DELAY_imputed,ARR_DELAY_imputed,DEP_DELAY_NEW_imputed,DISTANCE_imputed,TAXI_OUT_imputed,WEATHER_DELAY_imputed
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,172,172,543,543,0,543,1373,1373,1373,1373,0,0,0,0,1233,1233,0,0,0,341223,341223,341223,341223,341223,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
#encoder for categorical features
encoder = encoder_categorical_features(airline_delays_train, data_types_map)

root
-- YEAR: double (nullable = true)
-- MONTH: double (nullable = true)
-- DAY_OF_MONTH: double (nullable = true)
-- DAY_OF_WEEK: double (nullable = true)
-- FL_DATE: timestamp (nullable = true)
-- UNIQUE_CARRIER: string (nullable = false)
-- TAIL_NUM: string (nullable = false)
-- FL_NUM: double (nullable = true)
-- ORIGIN_AIRPORT_ID: double (nullable = true)
-- ORIGIN: string (nullable = false)
-- ORIGIN_CITY_NAME: string (nullable = false)
-- ORIGIN_STATE_ABR: string (nullable = false)
-- DEST_AIRPORT_ID: double (nullable = true)
-- DEST: string (nullable = false)
-- DEST_CITY_NAME: string (nullable = false)
-- DEST_STATE_ABR: string (nullable = false)
-- CRS_DEP_TIME: double (nullable = true)
-- DEP_TIME: double (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: double (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: double (nullable = true)
-- WHEELS_ON: double (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: double (nullable = true)
-- ARR_TIME: double (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: double (nullable = true)
-- CANCELLED: double (nullable = true)
-- CANCELLATION_CODE: string (nullable = false)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: double (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = true)
-- LATE_AIRCRAFT_DELAY: double (nullable = true)
-- _c46: string (nullable = false)
-- TAIL_NUM_index: double (nullable = false)
-- ORIGIN_index: double (nullable = false)
-- DEST_STATE_ABR_index: double (nullable = false)
-- CANCELLATION_CODE_index: double (nullable = false)
-- ORIGIN_STATE_ABR_index: double (nullable = false)
-- DEST_CITY_NAME_index: double (nullable = false)
-- DEST_index: double (nullable = false)
-- UNIQUE_CARRIER_index: double (nullable = false)
-- ORIGIN_CITY_NAME_index: double (nullable = false)
-- ORIGIN_STATE_ABR_encoded: vector (nullable = true)
-- TAIL_NUM_encoded: vector (nullable = true)
-- UNIQUE_CARRIER_encoded: vector (nullable = true)
-- DEST_encoded: vector (nullable = true)
-- CANCELLATION_CODE_encoded: vector (nullable = true)
-- ORIGIN_encoded: vector (nullable = true)
-- DEST_CITY_NAME_encoded: vector (nullable = true)
-- ORIGIN_CITY_NAME_encoded: vector (nullable = true)
-- DEST_STATE_ABR_encoded: vector (nullable = true)

In [16]:
#assembler for vector data
assembler = assembler_for_feature_vector(airline_delays_train, data_types_map)

['DEP_DELAY_NEW_imputed', 'TAXI_IN_imputed', 'ARR_DELAY_NEW_imputed', 'CANCELLED_imputed', 'DISTANCE_imputed', 'LATE_AIRCRAFT_DELAY_imputed', 'FLIGHTS_imputed', 'CRS_ELAPSED_TIME_imputed', 'ARR_DEL15_imputed', 'ARR_DELAY_imputed', 'CARRIER_DELAY_imputed', 'DIVERTED_imputed', 'ACTUAL_ELAPSED_TIME_imputed', 'TAXI_OUT_imputed', 'WEATHER_DELAY_imputed', 'AIR_TIME_imputed', 'SECURITY_DELAY_imputed', 'DEP_DELAY_imputed', 'NAS_DELAY_imputed']

In [17]:
pipeline = Pipeline(stages=imputer + encoder + assembler)
tmp = pipeline.fit(airline_delays_train).transform(airline_delays_train)
tmp.printSchema()

root
-- YEAR: double (nullable = true)
-- MONTH: double (nullable = true)
-- DAY_OF_MONTH: double (nullable = true)
-- DAY_OF_WEEK: double (nullable = true)
-- FL_DATE: timestamp (nullable = true)
-- UNIQUE_CARRIER: string (nullable = false)
-- TAIL_NUM: string (nullable = false)
-- FL_NUM: double (nullable = true)
-- ORIGIN_AIRPORT_ID: double (nullable = true)
-- ORIGIN: string (nullable = false)
-- ORIGIN_CITY_NAME: string (nullable = false)
-- ORIGIN_STATE_ABR: string (nullable = false)
-- DEST_AIRPORT_ID: double (nullable = true)
-- DEST: string (nullable = false)
-- DEST_CITY_NAME: string (nullable = false)
-- DEST_STATE_ABR: string (nullable = false)
-- CRS_DEP_TIME: double (nullable = true)
-- DEP_TIME: double (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: double (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: double (nullable = true)
-- WHEELS_ON: double (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: double (nullable = true)
-- ARR_TIME: double (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: double (nullable = true)
-- CANCELLED: double (nullable = true)
-- CANCELLATION_CODE: string (nullable = false)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: double (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = true)
-- LATE_AIRCRAFT_DELAY: double (nullable = true)
-- _c46: string (nullable = false)
-- ACTUAL_ELAPSED_TIME_imputed: double (nullable = true)
-- AIR_TIME_imputed: double (nullable = true)
-- CANCELLED_imputed: double (nullable = true)
-- ARR_DELAY_NEW_imputed: double (nullable = true)
-- FLIGHTS_imputed: double (nullable = true)
-- SECURITY_DELAY_imputed: double (nullable = true)
-- DEP_DELAY_imputed: double (nullable = true)
-- ARR_DEL15_imputed: double (nullable = true)
-- NAS_DELAY_imputed: double (nullable = true)
-- DIVERTED_imputed: double (nullable = true)
-- TAXI_IN_imputed: double (nullable = true)
-- CRS_ELAPSED_TIME_imputed: double (nullable = true)
-- CARRIER_DELAY_imputed: double (nullable = true)
-- LATE_AIRCRAFT_DELAY_imputed: double (nullable = true)
-- ARR_DELAY_imputed: double (nullable = true)
-- DEP_DELAY_NEW_imputed: double (nullable = true)
-- DISTANCE_imputed: double (nullable = true)
-- TAXI_OUT_imputed: double (nullable = true)
-- WEATHER_DELAY_imputed: double (nullable = true)
-- TAIL_NUM_index: double (nullable = false)
-- ORIGIN_index: double (nullable = false)
-- DEST_STATE_ABR_index: double (nullable = false)
-- CANCELLATION_CODE_index: double (nullable = false)
-- ORIGIN_STATE_ABR_index: double (nullable = false)
-- DEST_CITY_NAME_index: double (nullable = false)
-- DEST_index: double (nullable = false)
-- UNIQUE_CARRIER_index: double (nullable = false)
-- ORIGIN_CITY_NAME_index: double (nullable = false)
-- ORIGIN_STATE_ABR_encoded: vector (nullable = true)
-- TAIL_NUM_encoded: vector (nullable = true)
-- UNIQUE_CARRIER_encoded: vector (nullable = true)
-- DEST_encoded: vector (nullable = true)
-- CANCELLATION_CODE_encoded: vector (nullable = true)
-- ORIGIN_encoded: vector (nullable = true)
-- DEST_CITY_NAME_encoded: vector (nullable = true)
-- ORIGIN_CITY_NAME_encoded: vector (nullable = true)
-- DEST_STATE_ABR_encoded: vector (nullable = true)
-- features: vector (nullable = true)

In [18]:
#define the estimator
lr = LogisticRegression(featuresCol='features', labelCol = target)

# define the modeling pipeline with formula + feature transofrmations + estimator
pipeline = Pipeline(stages=imputer + encoder + assembler + [lr])

#define binary classification evaluator with right metric
evaluator = BinaryClassificationEvaluator(labelCol=target, metricName="areaUnderROC")

# Define the parameter grid for random forest
param_grid = ParamGridBuilder() \
             .addGrid(lr.regParam, [0.01, 0.5]) \
             .addGrid(lr.elasticNetParam, [0.5, 1.0]) \
             .addGrid(lr.maxIter, [1, 5]) \
             .build()

cv_model = build_and_tune_model_with_cv(pipeline, param_grid, evaluator, airline_delays_train)


In [19]:
model_summary_lr(cv_model)

CrossValidatorModel_4a16bde3c82bc5b2daa4
[0.986315130525322, 0.8663068103499796, 0.9865529454840931, 0.5, 0.9811282927279399, 0.8663068103499796, 0.9838223880009178, 0.5]
PipelineModel_42988008bc7d2acd5a72
[Imputer_43c583027a94204ae9e6, StringIndexer_402293c79e1706b5d94f, StringIndexer_4fdd95fdb8216877a12b, StringIndexer_4b5fabd24483dcbc2106, StringIndexer_4fa6bbbaf5bcab9a8930, StringIndexer_49f3b82da727084135fd, StringIndexer_4e81945a6bc94f96950a, StringIndexer_4770be57785317d0ce3c, StringIndexer_43b5b33f3a66c78df152, StringIndexer_48039f8e75a9be58f6f7, OneHotEncoderEstimator_46b9ace0a12c8e686bca, VectorAssembler_4439b54dff38020a6677, LogisticRegression_4b67bdc895eb27b92308]
LogisticRegression_4b67bdc895eb27b92308
[0.000932378635607,0.0,0.000852151912997,0.0,0.0,0.000328751438204,0.0,0.0,0.145086429169,0.000865613189812,0.000124774202051,0.0,0.0,0.000329725876603,1.5583546058e-05,0.0,0.0,0.00094495726381,-0.000240453941631]
-1.5179767588968156

In [20]:
predict(cv_model, evaluator, airline_delays_test)

DataFrame[YEAR: double, MONTH: double, DAY_OF_MONTH: double, DAY_OF_WEEK: double, FL_DATE: timestamp, UNIQUE_CARRIER: string, TAIL_NUM: string, FL_NUM: double, ORIGIN_AIRPORT_ID: double, ORIGIN: string, ORIGIN_CITY_NAME: string, ORIGIN_STATE_ABR: string, DEST_AIRPORT_ID: double, DEST: string, DEST_CITY_NAME: string, DEST_STATE_ABR: string, CRS_DEP_TIME: double, DEP_TIME: double, DEP_DELAY: double, DEP_DELAY_NEW: double, DEP_DEL15: double, DEP_DELAY_GROUP: double, TAXI_OUT: double, WHEELS_OFF: double, WHEELS_ON: double, TAXI_IN: double, CRS_ARR_TIME: double, ARR_TIME: double, ARR_DELAY: double, ARR_DELAY_NEW: double, ARR_DEL15: double, ARR_DELAY_GROUP: double, CANCELLED: double, CANCELLATION_CODE: string, DIVERTED: double, CRS_ELAPSED_TIME: double, ACTUAL_ELAPSED_TIME: double, AIR_TIME: double, FLIGHTS: double, DISTANCE: double, DISTANCE_GROUP: double, CARRIER_DELAY: double, WEATHER_DELAY: double, NAS_DELAY: double, SECURITY_DELAY: double, LATE_AIRCRAFT_DELAY: double, _c46: string, ACTUAL_ELAPSED_TIME_imputed: double, AIR_TIME_imputed: double, CANCELLED_imputed: double, ARR_DELAY_NEW_imputed: double, FLIGHTS_imputed: double, SECURITY_DELAY_imputed: double, DEP_DELAY_imputed: double, ARR_DEL15_imputed: double, NAS_DELAY_imputed: double, DIVERTED_imputed: double, TAXI_IN_imputed: double, CRS_ELAPSED_TIME_imputed: double, CARRIER_DELAY_imputed: double, LATE_AIRCRAFT_DELAY_imputed: double, ARR_DELAY_imputed: double, DEP_DELAY_NEW_imputed: double, DISTANCE_imputed: double, TAXI_OUT_imputed: double, WEATHER_DELAY_imputed: double, TAIL_NUM_index: double, ORIGIN_index: double, DEST_STATE_ABR_index: double, CANCELLATION_CODE_index: double, ORIGIN_STATE_ABR_index: double, DEST_CITY_NAME_index: double, DEST_index: double, UNIQUE_CARRIER_index: double, ORIGIN_CITY_NAME_index: double, ORIGIN_STATE_ABR_encoded: vector, TAIL_NUM_encoded: vector, UNIQUE_CARRIER_encoded: vector, DEST_encoded: vector, CANCELLATION_CODE_encoded: vector, ORIGIN_encoded: vector, DEST_CITY_NAME_encoded: vector, ORIGIN_CITY_NAME_encoded: vector, DEST_STATE_ABR_encoded: vector, features: vector, rawPrediction: vector, probability: vector, prediction: double]
root
-- YEAR: double (nullable = true)
-- MONTH: double (nullable = true)
-- DAY_OF_MONTH: double (nullable = true)
-- DAY_OF_WEEK: double (nullable = true)
-- FL_DATE: timestamp (nullable = true)
-- UNIQUE_CARRIER: string (nullable = true)
-- TAIL_NUM: string (nullable = true)
-- FL_NUM: double (nullable = true)
-- ORIGIN_AIRPORT_ID: double (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- DEST_AIRPORT_ID: double (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- CRS_DEP_TIME: double (nullable = true)
-- DEP_TIME: double (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: double (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: double (nullable = true)
-- WHEELS_ON: double (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: double (nullable = true)
-- ARR_TIME: double (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: double (nullable = true)
-- CANCELLED: double (nullable = true)
-- CANCELLATION_CODE: string (nullable = true)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: double (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = tru

In [21]:
persist_model(cv_model, "lr", "/FileStore/tables/")

In [22]:
loaded_model = load_model("/FileStore/tables/cv_model_lr_10-06-2018-1538862687")
print(loaded_model)
predict(loaded_model, evaluator, airline_delays_test)